<a href="https://colab.research.google.com/github/samiul272/ColabNotebooks/blob/master/cyclegan_xray.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install --upgrade tensorflow
!pip install -q  --no-deps tensorflow-addons~=0.6

     |████████████████████████████████| 86.3MB 117kB/s 
     |████████████████████████████████| 3.8MB 42.4MB/s 
     |████████████████████████████████| 450kB 42.2MB/s 
     |████████████████████████████████| 81kB 10.8MB/s 
ERROR: tensorboard 2.0.1 has requirement grpcio>=1.24.3, but you'll have grpcio 1.15.0 which is incompatible.
ERROR: google-colab 1.0.0 has requirement google-auth~=1.4.0, but you'll have google-auth 1.7.1 which is incompatible.
  Found existing installation: google-auth 1.4.2
    Uninstalling google-auth-1.4.2:
      Successfully uninstalled google-auth-1.4.2
  Found existing installation: tensorboard 1.15.0
    Uninstalling tensorboard-1.15.0:
      Successfully uninstalled tensorboard-1.15.0
  Found existing installation: tensorflow-estimator 1.15.1
    Uninstalling tensorflow-estimator-1.15.1:
      Successfully uninstalled tensorflow-estimator-1.15.1
  Found existing installation: tensorflow 1.15.0
    Uninstalling tensorflow-1.15.0:
      Successfully uninstall

     |████████████████████████████████| 1.8MB 7.0MB/s 


In [0]:
import os
api_token = {"username":"samiul272","key":"cf6765e49d37b7384d67b3298fd047e7"}
os.environ['KAGGLE_USERNAME'] = "samiul272"
os.environ['KAGGLE_KEY'] = "cf6765e49d37b7384d67b3298fd047e7"
import tqdm
!kaggle datasets download -d kmader/pulmonary-chest-xray-abnormalities

100% 4.07G/4.09G [00:36<00:00, 124MB/s]
100% 4.09G/4.09G [00:36<00:00, 121MB/s]


In [0]:
import numpy  as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.style  as style
import tensorflow as tf

from tqdm  import tqdm
import sklearn as sk
from sklearn.metrics import accuracy_score, roc_auc_score
from tensorflow.keras import layers, models, callbacks, activations, applications, optimizers
from tensorflow.keras.preprocessing import image
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [0]:
from __future__ import print_function, division
import scipy

from tensorflow.keras.datasets import mnist
import tensorflow_addons as tfa
from tensorflow.keras import *
import datetime
import matplotlib.pyplot as plt
import sys
import numpy as np
import os

In [0]:
!unzip pulmonary-chest-xray-abnormalities.zip -d data

Archive:  pulmonary-chest-xray-abnormalities.zip
  inflating: data/ChinaSet_AllFiles/ChinaSet_AllFiles/CXR_png/CHNCXR_0001_0.png  
  inflating: data/ChinaSet_AllFiles/ChinaSet_AllFiles/CXR_png/CHNCXR_0002_0.png  
  inflating: data/ChinaSet_AllFiles/ChinaSet_AllFiles/CXR_png/CHNCXR_0003_0.png  
  inflating: data/ChinaSet_AllFiles/ChinaSet_AllFiles/CXR_png/CHNCXR_0004_0.png  
  inflating: data/ChinaSet_AllFiles/ChinaSet_AllFiles/CXR_png/CHNCXR_0005_0.png  
  inflating: data/ChinaSet_AllFiles/ChinaSet_AllFiles/CXR_png/CHNCXR_0006_0.png  
  inflating: data/ChinaSet_AllFiles/ChinaSet_AllFiles/CXR_png/CHNCXR_0007_0.png  
  inflating: data/ChinaSet_AllFiles/ChinaSet_AllFiles/CXR_png/CHNCXR_0008_0.png  
  inflating: data/ChinaSet_AllFiles/ChinaSet_AllFiles/CXR_png/CHNCXR_0009_0.png  
  inflating: data/ChinaSet_AllFiles/ChinaSet_AllFiles/CXR_png/CHNCXR_0010_0.png  
  inflating: data/ChinaSet_AllFiles/ChinaSet_AllFiles/CXR_png/CHNCXR_0011_0.png  
  inflating: data/ChinaSet_AllFiles/ChinaSet_AllF

In [0]:
import glob
filelist_montgommery = glob.glob('data/Montgomery/MontgomerySet/CXR_png/*.png')
filelist_shenzen     = glob.glob('data/ChinaSet_AllFiles/ChinaSet_AllFiles/CXR_png/*.png')
import re
def extract_label(file_list):
    labels = []
    for file in tqdm(file_list):
        current_label = re.findall('[0-9]{4}_(.+?).png', file)
        labels.append(current_label[0])
    return(labels)
labels_montgommery = extract_label(filelist_montgommery)
labels_shenzen = extract_label(filelist_shenzen)
tdf = pd.DataFrame(filelist_shenzen, columns=['filepath'])
tdf['target'] = labels_shenzen
vdf = pd.DataFrame(filelist_montgommery, columns=['filepath'])
vdf['target'] = labels_montgommery

100%|██████████| 662/662 [00:00<00:00, 287465.50it/s]


In [0]:
batch_size = 16
input_size = (224,224)

atrain_df, aval_df = train_test_split(tdf,
                                    test_size    = 0.2,
                                    random_state = 451)

atrain_generator = ImageDataGenerator(rescale = 1./255,
                                     horizontal_flip = True,
                                     zoom_range      = 0.1,
                                     shear_range     = 0,
                                     rotation_range  = 5,
                                     width_shift_range  = 0.05,
                                     height_shift_range = 0.05,
                                     fill_mode = 'constant',
                                     cval      = 0)

atrain = atrain_generator.flow_from_dataframe(dataframe = atrain_df,
                                    class_mode  = 'binary',
                                    x_col       = 'filepath',
                                    y_col       = 'target',
                                    shuffle     = True,
                                    batch_size  = batch_size,
                                    target_size = input_size,
                                    seed=451)

aval = atrain_generator.flow_from_dataframe(dataframe = aval_df,
                                         class_mode  = 'binary',
                                         x_col       = 'filepath',
                                         y_col       = 'target',
                                         shuffle     = True,
                                         batch_size  = batch_size,
                                         target_size = input_size,
                                         seed=451)

btrain_df, bval_df = train_test_split(vdf,
                                    test_size    = 0.3,
                                    random_state = 451)

btrain_generator = ImageDataGenerator(rescale = 1./255,
                                     horizontal_flip = True,
                                     zoom_range      = 0.1,
                                     shear_range     = 0,
                                     rotation_range  = 5,
                                     width_shift_range  = 0.05,
                                     height_shift_range = 0.05,
                                     fill_mode = 'constant',
                                     cval      = 0)

btrain = btrain_generator.flow_from_dataframe(dataframe = btrain_df,
                                    class_mode  = 'binary',
                                    x_col       = 'filepath',
                                    y_col       = 'target',
                                    shuffle     = True,
                                    batch_size  = batch_size,
                                    target_size = input_size,
                                    seed=451)

aval = btrain_generator.flow_from_dataframe(dataframe = bval_df,
                                         class_mode  = 'binary',
                                         x_col       = 'filepath',
                                         y_col       = 'target',
                                         shuffle     = True,
                                         batch_size  = batch_size,
                                         target_size = input_size,
                                         seed=451)

Found 529 validated image filenames belonging to 2 classes.
Found 133 validated image filenames belonging to 2 classes.
Found 96 validated image filenames belonging to 2 classes.
Found 42 validated image filenames belonging to 2 classes.


In [0]:
BATCH_SIZE = 16
IMG_HEIGHT = 224
IMG_WIDTH = 224
AUTOTUNE = tf.data.experimental.AUTOTUNE

def decode_img(img):
    img = tf.image.decode_jpeg(img, channels=1)
    img = tf.image.convert_image_dtype(img, tf.float32)
    return tf.image.resize(img, [IMG_WIDTH, IMG_HEIGHT])
def process_inp(file_path, label):
    img = tf.io.read_file(file_path)
    img = decode_img(img)
    return img, label

def augment_img(img, label):
    img = tf.image.random_brightness(img, 0.2)
    img = tf.image.random_flip_left_right(img)
    img = tf.image.random_contrast(img, 0, 0.2)
    return img, label

def load_data(dataset):
    dataset = dataset.map(process_inp, num_parallel_calls=AUTOTUNE)
    dataset.cache()
    dataset = dataset.map(augment_img, num_parallel_calls=AUTOTUNE)
    return dataset

def prepare_for_training(a, b, shuffle_buffer_size=1000):
    ds = tf.data.Dataset.zip((a, b))
    ds = ds.shuffle(buffer_size=shuffle_buffer_size)
    ds = ds.repeat()
    ds = ds.batch(BATCH_SIZE)
    ds = ds.prefetch(buffer_size=AUTOTUNE)
    return ds

adataset = tf.data.Dataset.from_tensor_slices((atrain_df.filepath, atrain_df.target))
bdataset = tf.data.Dataset.from_tensor_slices((btrain_df.filepath, btrain_df.target))

adataset = load_data(adataset)
bdataset = load_data(bdataset)
ds = prepare_for_training(adataset, bdataset, shuffle_buffer_size=75)

In [0]:
def cgan_data_loader(num_steps):
    for i in range(num_steps):
        yield next(atrain), next(btrain)
gen = cgan_data_loader(20)

0 16 16


KeyboardInterrupt: ignored

In [0]:
y[1].shape

(32,)

In [0]:
from tensorboardcolab import *
tcb = TensorBoardColab()
writer = tf.summary.create_file_writer("./Graph")
!pip install --upgrade grpcio
%load_ext tensorboard
%tensorboard --logdir ./Graph

Using TensorFlow backend.


Wait for 8 seconds...
TensorBoard link:
https://b18ec67c.ngrok.io
     |████████████████████████████████| 2.4MB 6.8MB/s 
  Found existing installation: grpcio 1.15.0
    Uninstalling grpcio-1.15.0:
      Successfully uninstalled grpcio-1.15.0


In [0]:
class CycleGAN():
    def __init__(self):
        # Input shape
        self.img_rows = 224
        self.img_cols = 224
        self.channels = 1
        self.img_shape = (self.img_rows, self.img_cols, self.channels)

        # Configure data loader
        # self.dataset_name = 'apple2orange'
        # self.data_loader = DataLoader(dataset_name=self.dataset_name,
        #                               img_res=(self.img_rows, self.img_cols))


        # Calculate output shape of D (PatchGAN)
        patch = int(self.img_rows / 2**4)
        self.disc_patch = (patch, patch, 1)

        # Number of filters in the first layer of G and D
        self.gf = 32
        self.df = 64

        # Loss weights
        self.lambda_cycle = 10.0                    # Cycle-consistency loss
        self.lambda_id = 0.1 * self.lambda_cycle    # Identity loss

        optimizer = optimizers.Adam(0.0002, 0.5)

        # Build and compile the discriminators
        self.d_A = self.build_discriminator()
        self.d_B = self.build_discriminator()
        self.d_A.compile(loss='mse',
            optimizer=optimizer,
            metrics=['accuracy'])
        self.d_B.compile(loss='mse',
            optimizer=optimizer,
            metrics=['accuracy'])

        #-------------------------
        # Construct Computational
        #   Graph of Generators
        #-------------------------

        # Build the generators
        self.g_AB = self.build_generator()
        self.g_BA = self.build_generator()

        # Input images from both domains
        img_A = Input(shape=self.img_shape)
        img_B = Input(shape=self.img_shape)

        # Translate images to the other domain
        fake_B = self.g_AB(img_A)
        fake_A = self.g_BA(img_B)
        # Translate images back to original domain
        reconstr_A = self.g_BA(fake_B)
        reconstr_B = self.g_AB(fake_A)
        # Identity mapping of images
        img_A_id = self.g_BA(img_A)
        img_B_id = self.g_AB(img_B)

        # For the combined model we will only train the generators
        self.d_A.trainable = False
        self.d_B.trainable = False

        # Discriminators determines validity of translated images
        valid_A = self.d_A(fake_A)
        valid_B = self.d_B(fake_B)

        # Combined model trains generators to fool discriminators
        self.combined = models.Model(inputs=[img_A, img_B],
                              outputs=[ valid_A, valid_B,
                                        reconstr_A, reconstr_B,
                                        img_A_id, img_B_id ])
        self.combined.compile(loss=['mse', 'mse',
                                    'mae', 'mae',
                                    'mae', 'mae'],
                            loss_weights=[  1, 1,
                                            self.lambda_cycle, self.lambda_cycle,
                                            self.lambda_id, self.lambda_id ],
                            optimizer=optimizer)

    def build_generator(self):
        """U-Net Generator"""

        def conv2d(layer_input, filters, f_size=4):
            """Layers used during downsampling"""
            d = layers.Conv2D(filters, kernel_size=f_size, strides=2, padding='same')(layer_input)
            d = layers.LeakyReLU(alpha=0.2)(d)
            d = tfa.layers.InstanceNormalization()(d)
            return d

        def deconv2d(layer_input, skip_input, filters, f_size=4, dropout_rate=0):
            """Layers used during upsampling"""
            u = layers.UpSampling2D(size=2)(layer_input)
            u = layers.Conv2D(filters, kernel_size=f_size, strides=1, padding='same', activation='relu')(u)
            if dropout_rate:
                u = layers.Dropout(dropout_rate)(u)
            u = tfa.layers.InstanceNormalization()(u)
            u = layers.Concatenate()([u, skip_input])
            return u

        # Image input
        d0 = layers.Input(shape=self.img_shape)

        # Downsampling
        d1 = conv2d(d0, self.gf)
        d2 = conv2d(d1, self.gf*2)
        d3 = conv2d(d2, self.gf*4)
        d4 = conv2d(d3, self.gf*8)

        # Upsampling
        u1 = deconv2d(d4, d3, self.gf*4)
        u2 = deconv2d(u1, d2, self.gf*2)
        u3 = deconv2d(u2, d1, self.gf)

        u4 = layers.UpSampling2D(size=2)(u3)
        output_img = layers.Conv2D(self.channels, kernel_size=4, strides=1, padding='same', activation='tanh')(u4)

        return models.Model(d0, output_img)

    def build_discriminator(self):
        def d_layer(layer_input, filters, f_size=4, normalization=True):
            """Discriminator layer"""
            d = layers.Conv2D(filters, kernel_size=f_size, strides=2, padding='same')(layer_input)
            d = layers.LeakyReLU(alpha=0.2)(d)
            if normalization:
                d = tfa.layers.InstanceNormalization()(d)
            return d

        img = layers.Input(shape=self.img_shape)

        d1 = d_layer(img, self.df, normalization=False)
        d2 = d_layer(d1, self.df*2)
        d3 = d_layer(d2, self.df*4)
        d4 = d_layer(d3, self.df*8)

        validity = layers.Conv2D(1, kernel_size=4, strides=1, padding='same')(d4)

        return models.Model(img, validity)
    # build_discriminator(layers.Input((256,256,3))).summary()

    def train(self, epochs, batch_size=16, sample_interval=5):

        start_time = datetime.datetime.now()

        # Adversarial loss ground truths
        valid = np.ones((batch_size,) + self.disc_patch)
        fake = np.zeros((batch_size,) + self.disc_patch)

        for epoch in range(epochs):
            print(f"Epoch {epoch} of {epochs}")
            iterator = tqdm(enumerate(ds), total = 20)
            for batch_i, ((imgs_A, _), (imgs_B, _)) in iterator:
                # ----------------------
                #  Train Discriminators
                # ----------------------

                # Translate images to opposite domain
                fake_B = self.g_AB.predict(imgs_A, steps=1)
                fake_A = self.g_BA.predict(imgs_B, steps=1)

                reconstr_A = self.g_BA.predict(fake_B)
                # Train the discriminators (original images = real / translated = Fake)
                dA_loss_real = self.d_A.train_on_batch(imgs_A, valid)
                dA_loss_fake = self.d_A.train_on_batch(fake_A, fake)
                dA_loss = 0.5 * np.add(dA_loss_real, dA_loss_fake)

                dB_loss_real = self.d_B.train_on_batch(imgs_B, valid)
                dB_loss_fake = self.d_B.train_on_batch(fake_B, fake)
                dB_loss = 0.5 * np.add(dB_loss_real, dB_loss_fake)

                # Total disciminator loss
                d_loss = 0.5 * np.add(dA_loss, dB_loss)


                # ------------------
                #  Train Generators
                # ------------------

                # Train the generators
                g_loss = self.combined.train_on_batch([imgs_A, imgs_B],
                                                        [valid, valid,
                                                        imgs_A, imgs_B,
                                                        imgs_A, imgs_B])

                elapsed_time = datetime.datetime.now() - start_time

                # Plot the progress
                iterator.set_description(f"[Batch {batch_i}/{16}] "+
                                         f"[D loss: {d_loss[0]:.3f}, acc: {100*d_loss[1]:.3f}] "+
                                         f"[G loss: {g_loss[0]:.3f}, adv: {np.mean(g_loss[1:3]):.3f}, "+
                                         f"recon: {np.mean(g_loss[3:5]):.3f}, "+
                                         f"id: {np.mean(g_loss[5:6]):.3f}] "+
                                         f"time: {elapsed_time}")

                if batch_i % sample_interval == 0:
                    with writer.as_default():
                        tf.summary.scalar('Generator Loss', g_loss[0], epoch*25+batch_i)
                        tf.summary.scalar('Discriminator Loss', d_loss[0], epoch*25+batch_i)
                        tf.summary.scalar('Adv Loss', np.mean(g_loss[1:3]), epoch*25+batch_i)                    
                        tf.summary.scalar('Recon Loss', np.mean(g_loss[3:5]), epoch*25+batch_i)
                        tf.summary.scalar('Generator Acc', 100*d_loss[1], epoch*25+batch_i)
                        tf.summary.scalar('Identity loss', np.mean(g_loss[5:6]), epoch*25+batch_i)
                        tf.summary.image("Original examples", imgs_A, max_outputs=4, step=epoch*25+batch_i)
                        tf.summary.image("Translated examples", fake_B, max_outputs=4, step=epoch*25+batch_i)
                        tf.summary.image("Reconstructed examples", reconstr_A, max_outputs=4, step=epoch*25+batch_i)
                    
                    writer.flush()
                # If at save interval => save generated image samples
                # if batch_i % sample_interval == 0:
                #     self.sample_images(epoch, batch_i)
gan = CycleGAN()

In [0]:
gan.train(5)





  0%|          | 0/20 [00:00<?, ?it/s]

Epoch 0 of 5






[Batch 0/16] [D loss: 16.016, acc: 65.075] [G loss: 163.053, adv: 71.458, recon: 0.908, id: 0.738] time: 0:02:51.317650:   0%|          | 0/20 [02:51<?, ?it/s]



[Batch 0/16] [D loss: 16.016, acc: 65.075] [G loss: 163.053, adv: 71.458, recon: 0.908, id: 0.738] time: 0:02:51.317650:   5%|▌         | 1/20 [02:51<54:16, 171.38s/it]



[Batch 1/16] [D loss: 97.696, acc: 56.505] [G loss: 16.131, adv: 3.640, recon: 0.400, id: 0.376] time: 0:05:12.790789:   5%|▌         | 1/20 [05:12<54:16, 171.38s/it]  



[Batch 1/16] [D loss: 97.696, acc: 56.505] [G loss: 16.131, adv: 3.640, recon: 0.400, id: 0.376] time: 0:05:12.790789:  10%|█         | 2/20 [05:12<48:42, 162.38s/it]



[Batch 2/16] [D loss: 7.292, acc: 50.136] [G loss: 7.555, adv: 0.668, recon: 0.279, id: 0.369] time: 0:07:33.767118:  10%|█         | 2/20 [07:33<48:42, 162.38s/it]  



[Batch 2/16] [D loss: 7.292, acc: 50.136] [G loss: 7.555, adv: 0.668, recon: 0.279, id: 0.369] time: 0:07:33.767118:  15%|█▌        | 3/20 [07:33<44:

In [0]:
valid

NameError: ignored